In [1]:
##Import packages needed (and some not needed....)

import Bio

import openpyxl
from Bio import SeqIO
import os,sys,re

import pandas as pd
print(pd.__name__, pd.__version__)

import numpy as np
print(np.__name__, np.__version__)

import matplotlib.pyplot as plt
import matplotlib
print(matplotlib.__name__, matplotlib.__version__)
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import MaxNLocator

import scipy.stats as stats
import scipy
print(scipy.__name__, scipy.__version__)

import seaborn as sns
print(sns.__name__, sns.__version__)

from collections import Counter

import scipy as sp
import math
from scipy.stats import shapiro 
from scipy.stats import lognorm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_rel

pandas 1.5.2
numpy 1.23.5
matplotlib 3.4.1
scipy 1.10.0
seaborn 0.12.2


In [5]:
#open df with number of reads
df_none = pd.read_csv("../Nothing/Variant_count_Lanes_summed_No Antifungal_NovaSeq_winter2023_albicans_2023-12-07.csv")
df_fluco = pd.read_csv("../Fluconazole/Variant_count_Lanes_summed_Fluconazole_NovaSeq_winter2023_albicans_2023-12-06.csv")
df_vorico = pd.read_csv("../Voriconazole/Variant_count_Lanes_summed_Voriconazole_NovaSeq_winter2023_albicans_2023-12-06.csv")
df_itra = pd.read_csv("../Itraconazole/Variant_count_Lanes_summed_Itraconazole_NovaSeq_winter2023_albicans_2023-12-06.csv")
df_posaco = pd.read_csv("../Posaconazole/Variant_count_Lanes_summed_Posaconazole_NovaSeq_winter2023_albicans_2023-12-06.csv")
df_isa = pd.read_csv("../Isavuconazole/Variant_count_Lanes_summed_Isavuconazole_NovaSeq_winter2023_albicans_2023-12-06.csv")
df_clotri = pd.read_csv("../Clotrimazole/Variant_count_Lanes_summed_Clotrimazole_NovaSeq_winter2023_albicans_2023-12-06.csv")

In [6]:
# Create a list of DataFrames to concatenate
dfs_to_concatenate = [df_none, df_fluco, df_vorico, df_itra, df_posaco, df_isa, df_clotri]

# Use pd.concat to concatenate the DataFrames vertically
df_v = pd.concat(dfs_to_concatenate, ignore_index=True)
columns_to_keep = ["Condition", "position", "codon", "Timepoint", "Antifungal", "Fragment", "Replicate", "nbr_reads","nbr_reads_log"]
df_v = df_v[columns_to_keep]

#remove failed samples
mask = (df_v['Antifungal'] != 'Vorico') | (df_v['Fragment'] != 'F1') | (df_v['Replicate'] != 'A')
# Apply the mask to filter the DataFrame
df_v = df_v[mask]

#remove failed samples
mask = (df_v['Antifungal'] != 'Itra') | (df_v['Fragment'] != 'F4') | (df_v['Replicate'] != 'C')
# Apply the mask to filter the DataFrame
df_v = df_v[mask]

replicate_mapping = {1: 'A', 2: 'B', 3: 'C'}

# Use the replace method to replace the values in the 'Replicate' column
df_v['Replicate'] = df_v['Replicate'].replace(replicate_mapping)

df_v

,Condition,position,codon,Timepoint,Antifungal,Fragment,Replicate,nbr_reads,nbr_reads_log
0,CaERG11_0_None_F1_1,55,AAA,0,None,F1,A,1230.0,10.265615
1,CaERG11_0_None_F1_1,55,AAC,0,None,F1,A,903.0,9.820179
2,CaERG11_0_None_F1_1,55,AAG,0,None,F1,A,1292.0,10.336507
3,CaERG11_0_None_F1_1,55,AAT,0,None,F1,A,726.0,9.505812
4,CaERG11_0_None_F1_1,55,ACC,0,None,F1,A,678.0,9.407268
...,...,...,...,...,...,...,...,...,...
335659,CaERG11_2_Clotri_F4_C,511,TGT,2,Clotri,F4,C,656.0,9.359750
335660,CaERG11_2_Clotri_F4_C,511,TTA,2,Clotri,F4,C,359.0,8.491853
335661,CaERG11_2_Clotri_F4_C,511,TTC,2,Clotri,F4,C,3787.0,11.887221
335662,CaERG11_2_Clotri_F4_C,511,TTG,2,Clotri,F4,C,472.0,8.885696


In [15]:
print(df_v[df_v["Timepoint"] == 0]["nbr_reads"].mean())
print(df_v[df_v["Timepoint"] == 2]["nbr_reads"].mean())
df_v[df_v["Timepoint"] == 3]["nbr_reads"].mean()

748.5714811520412
716.7383539870108


987.5505505505505

In [17]:
print(df_v[df_v["Timepoint"] == 0]["nbr_reads"].median())
print(df_v[df_v["Timepoint"] == 2]["nbr_reads"].median())
df_v[df_v["Timepoint"] == 3]["nbr_reads"].median()

661.0
199.0


958.0